In [6]:
from urllib.request import urlopen
from zipfile import ZipFile
import io
import shapefile
import geopandas as gpd
from shapely.geometry import shape  
import pandas as pd
import requests
import matplotlib.pyplot as plt

In [69]:
df_census = pd.read_pickle('df_census.pkl')
df_census

,GEO_ID,NAME,Total - AGE BY EDUCATIONAL ATTAINMENT - Population 25 years and over,Total - AGE BY EDUCATIONAL ATTAINMENT - Population 25 years and over - Bachelor's degree or higher,Total - MEDIAN EARNINGS IN THE PAST 12 MONTHS (IN 2019 INFLATION-ADJUSTED DOLLARS) - Population 25 years and over with earnings,Female - AGE BY EDUCATIONAL ATTAINMENT - Population 25 years and over - Bachelor's degree,Female - AGE BY EDUCATIONAL ATTAINMENT - Population 25 years and over - Graduate or professional degree,Female - MEDIAN EARNINGS IN THE PAST 12 MONTHS (IN 2019 INFLATION-ADJUSTED DOLLARS) - Population 25 years and over with earnings - Bachelor's degree,Female - MEDIAN EARNINGS IN THE PAST 12 MONTHS (IN 2019 INFLATION-ADJUSTED DOLLARS) - Population 25 years and over with earnings - Graduate or professional degree,Total population - Sex ratio (males per 100 females),Female Ratio,Total Female Population - 25 years and over
1,36061000100,"Census Tract 1, New York County, New York",0,0,0,0,0,0,0,0.0,1.000000,0.000000
2,36061000201,"Census Tract 2.01, New York County, New York",1871,310,29496,153,54,50789,0,73.7,0.575705,1077.144502
3,36061000202,"Census Tract 2.02, New York County, New York",5945,2021,42013,667,516,42311,69280,77.7,0.562746,3345.526168
4,36061000500,"Census Tract 5, New York County, New York",0,0,0,0,0,0,0,0.0,1.000000,0.000000
5,36061000600,"Census Tract 6, New York County, New York",8013,1408,27147,599,187,0,113571,87.6,0.533049,4271.321962
...,...,...,...,...,...,...,...,...,...,...,...,...
284,36061030900,"Census Tract 309, New York County, New York",6962,1373,26041,671,244,41827,59432,81.9,0.549753,3827.377680
285,36061031100,"Census Tract 311, New York County, New York",0,0,0,0,0,0,0,0.0,1.000000,0.000000
286,36061031703,"Census Tract 317.03, New York County, New York",3765,3103,157464,701,820,110313,118426,100.1,0.499750,1881.559220
287,36061031704,"Census Tract 317.04, New York County, New York",7893,7119,128162,1746,1876,85389,128389,106.3,0.484731,3825.981580


In [70]:
#Rename the GEO_ID column into GEOID
df_census.rename(columns={'GEO_ID':'GEOID'}, inplace=True)
df_census

,GEOID,NAME,Total - AGE BY EDUCATIONAL ATTAINMENT - Population 25 years and over,Total - AGE BY EDUCATIONAL ATTAINMENT - Population 25 years and over - Bachelor's degree or higher,Total - MEDIAN EARNINGS IN THE PAST 12 MONTHS (IN 2019 INFLATION-ADJUSTED DOLLARS) - Population 25 years and over with earnings,Female - AGE BY EDUCATIONAL ATTAINMENT - Population 25 years and over - Bachelor's degree,Female - AGE BY EDUCATIONAL ATTAINMENT - Population 25 years and over - Graduate or professional degree,Female - MEDIAN EARNINGS IN THE PAST 12 MONTHS (IN 2019 INFLATION-ADJUSTED DOLLARS) - Population 25 years and over with earnings - Bachelor's degree,Female - MEDIAN EARNINGS IN THE PAST 12 MONTHS (IN 2019 INFLATION-ADJUSTED DOLLARS) - Population 25 years and over with earnings - Graduate or professional degree,Total population - Sex ratio (males per 100 females),Female Ratio,Total Female Population - 25 years and over
1,36061000100,"Census Tract 1, New York County, New York",0,0,0,0,0,0,0,0.0,1.000000,0.000000
2,36061000201,"Census Tract 2.01, New York County, New York",1871,310,29496,153,54,50789,0,73.7,0.575705,1077.144502
3,36061000202,"Census Tract 2.02, New York County, New York",5945,2021,42013,667,516,42311,69280,77.7,0.562746,3345.526168
4,36061000500,"Census Tract 5, New York County, New York",0,0,0,0,0,0,0,0.0,1.000000,0.000000
5,36061000600,"Census Tract 6, New York County, New York",8013,1408,27147,599,187,0,113571,87.6,0.533049,4271.321962
...,...,...,...,...,...,...,...,...,...,...,...,...
284,36061030900,"Census Tract 309, New York County, New York",6962,1373,26041,671,244,41827,59432,81.9,0.549753,3827.377680
285,36061031100,"Census Tract 311, New York County, New York",0,0,0,0,0,0,0,0.0,1.000000,0.000000
286,36061031703,"Census Tract 317.03, New York County, New York",3765,3103,157464,701,820,110313,118426,100.1,0.499750,1881.559220
287,36061031704,"Census Tract 317.04, New York County, New York",7893,7119,128162,1746,1876,85389,128389,106.3,0.484731,3825.981580


In [71]:
#2 reading in our Census Tract Data shapefile as a Geopandas.dateframe
gpd2019shp = gpd.read_file("tl_2019_36_tract.shp")
gpd2019shp.crs

<Geographic 2D CRS: EPSG:4269>
Name: NAD83
Axis Info [ellipsoidal]:
- Lat[north]: Geodetic latitude (degree)
- Lon[east]: Geodetic longitude (degree)
Area of Use:
- name: North America - onshore and offshore: Canada - Alberta; British Columbia; Manitoba; New Brunswick; Newfoundland and Labrador; Northwest Territories; Nova Scotia; Nunavut; Ontario; Prince Edward Island; Quebec; Saskatchewan; Yukon. Puerto Rico. United States (USA) - Alabama; Alaska; Arizona; Arkansas; California; Colorado; Connecticut; Delaware; Florida; Georgia; Hawaii; Idaho; Illinois; Indiana; Iowa; Kansas; Kentucky; Louisiana; Maine; Maryland; Massachusetts; Michigan; Minnesota; Mississippi; Missouri; Montana; Nebraska; Nevada; New Hampshire; New Jersey; New Mexico; New York; North Carolina; North Dakota; Ohio; Oklahoma; Oregon; Pennsylvania; Rhode Island; South Carolina; South Dakota; Tennessee; Texas; Utah; Vermont; Virginia; Washington; West Virginia; Wisconsin; Wyoming. US Virgin Islands.  British Virgin Island

In [72]:
gpd2019shp

,STATEFP,COUNTYFP,TRACTCE,GEOID,NAME,NAMELSAD,MTFCC,FUNCSTAT,ALAND,AWATER,INTPTLAT,INTPTLON,geometry
0,36,081,044800,36081044800,448,Census Tract 448,G5020,S,208002,0,+40.7110219,-073.8026344,"POLYGON ((-73.80646 40.71206, -73.80556 40.712..."
1,36,081,045800,36081045800,458,Census Tract 458,G5020,S,245281,0,+40.7152626,-073.7909261,"POLYGON ((-73.79364 40.71382, -73.79362 40.713..."
2,36,081,046200,36081046200,462,Census Tract 462,G5020,S,249611,0,+40.7098547,-073.7879749,"POLYGON ((-73.79203 40.71107, -73.79101 40.711..."
3,36,081,046300,36081046300,463,Census Tract 463,G5020,S,159415,0,+40.7440007,-073.8710900,"POLYGON ((-73.87468 40.74335, -73.87423 40.743..."
4,36,081,046400,36081046400,464,Census Tract 464,G5020,S,228767,0,+40.7168637,-073.7869958,"POLYGON ((-73.79187 40.71379, -73.79085 40.714..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...
4913,36,081,027900,36081027900,279,Census Tract 279,G5020,S,177155,0,+40.7516729,-073.8802222,"POLYGON ((-73.88189 40.75208, -73.88096 40.752..."
4914,36,081,028000,36081028000,280,Census Tract 280,G5020,S,165398,0,+40.6883158,-073.7797191,"POLYGON ((-73.78243 40.69034, -73.78160 40.690..."
4915,36,081,028100,36081028100,281,Census Tract 281,G5020,S,174444,0,+40.7522203,-073.8825920,"POLYGON ((-73.88431 40.75567, -73.88351 40.755..."
4916,36,081,028200,36081028200,282,Census Tract 282,G5020,S,182472,0,+40.6849947,-073.7738873,"POLYGON ((-73.77709 40.68820, -73.77662 40.688..."


In [73]:
# Merging our dataframes together on Geo_ID tag
first_merge = gpd2019shp.merge(df_census, on='GEOID')
first_merge

,STATEFP,COUNTYFP,TRACTCE,GEOID,NAME_x,NAMELSAD,MTFCC,FUNCSTAT,ALAND,AWATER,...,Total - AGE BY EDUCATIONAL ATTAINMENT - Population 25 years and over,Total - AGE BY EDUCATIONAL ATTAINMENT - Population 25 years and over - Bachelor's degree or higher,Total - MEDIAN EARNINGS IN THE PAST 12 MONTHS (IN 2019 INFLATION-ADJUSTED DOLLARS) - Population 25 years and over with earnings,Female - AGE BY EDUCATIONAL ATTAINMENT - Population 25 years and over - Bachelor's degree,Female - AGE BY EDUCATIONAL ATTAINMENT - Population 25 years and over - Graduate or professional degree,Female - MEDIAN EARNINGS IN THE PAST 12 MONTHS (IN 2019 INFLATION-ADJUSTED DOLLARS) - Population 25 years and over with earnings - Bachelor's degree,Female - MEDIAN EARNINGS IN THE PAST 12 MONTHS (IN 2019 INFLATION-ADJUSTED DOLLARS) - Population 25 years and over with earnings - Graduate or professional degree,Total population - Sex ratio (males per 100 females),Female Ratio,Total Female Population - 25 years and over
0,36,061,000700,36061000700,7,Census Tract 7,G5020,S,253655,245812,...,6468,5803,113125,1869,1066,96724,102450,83.5,0.544959,3524.795640
1,36,061,021500,36061021500,215,Census Tract 215,G5020,S,69708,0,...,3013,912,39357,307,156,42969,93393,98.6,0.503525,1517.119839
2,36,061,025900,36061025900,259,Census Tract 259,G5020,S,75469,0,...,2711,1571,46972,488,314,52679,69000,85.2,0.539957,1463.822894
3,36,061,023801,36061023801,238.01,Census Tract 238.01,G5020,S,352467,518667,...,5214,3672,77782,774,1360,86369,86932,94.1,0.515198,2686.244204
4,36,061,031703,36061031703,317.03,Census Tract 317.03,G5020,S,248049,467876,...,3765,3103,157464,701,820,110313,118426,100.1,0.499750,1881.559220
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
283,36,061,015002,36061015002,150.02,Census Tract 150.02,G5020,S,125571,0,...,3777,3422,112417,729,1122,93958,111563,78.0,0.561798,2121.910112
284,36,061,015200,36061015200,152,Census Tract 152,G5020,S,189340,529192,...,5939,3999,77879,1277,1009,58269,80714,63.9,0.610128,3623.550946
285,36,061,019100,36061019100,191,Census Tract 191,G5020,S,223477,175654,...,7806,5124,90201,1164,1413,92225,112865,95.0,0.512821,4003.076923
286,36,061,031900,36061031900,319,Census Tract 319,G5020,S,99774,317234,...,0,0,0,0,0,0,0,0.0,1.000000,0.000000


In [75]:
#Ensure the crs(Coordinate Scale) matches set it to EPSG:4269
first_merge = first_merge.to_crs("EPSG:4269")
first_merge.crs

<Geographic 2D CRS: EPSG:4269>
Name: NAD83
Axis Info [ellipsoidal]:
- Lat[north]: Geodetic latitude (degree)
- Lon[east]: Geodetic longitude (degree)
Area of Use:
- name: North America - onshore and offshore: Canada - Alberta; British Columbia; Manitoba; New Brunswick; Newfoundland and Labrador; Northwest Territories; Nova Scotia; Nunavut; Ontario; Prince Edward Island; Quebec; Saskatchewan; Yukon. Puerto Rico. United States (USA) - Alabama; Alaska; Arizona; Arkansas; California; Colorado; Connecticut; Delaware; Florida; Georgia; Hawaii; Idaho; Illinois; Indiana; Iowa; Kansas; Kentucky; Louisiana; Maine; Maryland; Massachusetts; Michigan; Minnesota; Mississippi; Missouri; Montana; Nebraska; Nevada; New Hampshire; New Jersey; New Mexico; New York; North Carolina; North Dakota; Ohio; Oklahoma; Oregon; Pennsylvania; Rhode Island; South Carolina; South Dakota; Tennessee; Texas; Utah; Vermont; Virginia; Washington; West Virginia; Wisconsin; Wyoming. US Virgin Islands.  British Virgin Island

In [62]:
#3 reading in our GeoJSON file for MTA_Subway_Stations linking GPS coordinates from NYC Open Data
url_for_MTA = "https://data.cityofnewyork.us/api/geospatial/arq3-7z49?method=export&format=GeoJSON"
MTA_Subway_Stations = gpd.read_file(url_for_MTA)

In [63]:
MTA_Subway_Stations

,name,url,line,objectid,notes,geometry
0,Astor Pl,http://web.mta.info/nyct/service/,4-6-6 Express,1,"4 nights, 6-all times, 6 Express-weekdays AM s...",POINT (-73.99107 40.73005)
1,Canal St,http://web.mta.info/nyct/service/,4-6-6 Express,2,"4 nights, 6-all times, 6 Express-weekdays AM s...",POINT (-74.00019 40.71880)
2,50th St,http://web.mta.info/nyct/service/,1-2,3,"1-all times, 2-nights",POINT (-73.98385 40.76173)
3,Bergen St,http://web.mta.info/nyct/service/,2-3-4,4,"4-nights, 3-all other times, 2-all times",POINT (-73.97500 40.68086)
4,Pennsylvania Ave,http://web.mta.info/nyct/service/,3-4,5,"4-nights, 3-all other times",POINT (-73.89489 40.66471)
...,...,...,...,...,...,...
468,Coney Island - Stillwell Av,http://web.mta.info/nyct/service/,D-F-N-Q,469,"D,F,N,Q-all times",POINT (-73.98124 40.57728)
469,34th St - Hudson Yards,http://web.mta.info/nyct/service/,7-7 Express,470,"7-all times, 7 Express-rush hours AM westbound...",POINT (-74.00220 40.75545)
470,72nd St,http://web.mta.info/nyct/service/,Q,641,Q-all times,POINT (-73.95836 40.76880)
471,86th St,http://web.mta.info/nyct/service/,Q,642,Q-all times,POINT (-73.95177 40.77786)


In [76]:
#Before Spatial Joining we must check the crs settings for each file
MTA_Subway_Stations = MTA_Subway_Stations.to_crs("EPSG:4269")
MTA_Subway_Stations = MTA_Subway_Stations.to_crs(epsg=4269)
MTA_Subway_Stations.crs

<Geographic 2D CRS: EPSG:4269>
Name: NAD83
Axis Info [ellipsoidal]:
- Lat[north]: Geodetic latitude (degree)
- Lon[east]: Geodetic longitude (degree)
Area of Use:
- name: North America - onshore and offshore: Canada - Alberta; British Columbia; Manitoba; New Brunswick; Newfoundland and Labrador; Northwest Territories; Nova Scotia; Nunavut; Ontario; Prince Edward Island; Quebec; Saskatchewan; Yukon. Puerto Rico. United States (USA) - Alabama; Alaska; Arizona; Arkansas; California; Colorado; Connecticut; Delaware; Florida; Georgia; Hawaii; Idaho; Illinois; Indiana; Iowa; Kansas; Kentucky; Louisiana; Maine; Maryland; Massachusetts; Michigan; Minnesota; Mississippi; Missouri; Montana; Nebraska; Nevada; New Hampshire; New Jersey; New Mexico; New York; North Carolina; North Dakota; Ohio; Oklahoma; Oregon; Pennsylvania; Rhode Island; South Carolina; South Dakota; Tennessee; Texas; Utah; Vermont; Virginia; Washington; West Virginia; Wisconsin; Wyoming. US Virgin Islands.  British Virgin Island

In [77]:
#Spatial Join using geometry column in both gpd.dataframes, state 'intersect' or "within" 
spatialjoin = gpd.sjoin(MTA_Subway_Stations, first_merge, op='intersects')
spatialjoin

,name,url,line,objectid,notes,geometry,index_right,STATEFP,COUNTYFP,TRACTCE,...,Total - AGE BY EDUCATIONAL ATTAINMENT - Population 25 years and over,Total - AGE BY EDUCATIONAL ATTAINMENT - Population 25 years and over - Bachelor's degree or higher,Total - MEDIAN EARNINGS IN THE PAST 12 MONTHS (IN 2019 INFLATION-ADJUSTED DOLLARS) - Population 25 years and over with earnings,Female - AGE BY EDUCATIONAL ATTAINMENT - Population 25 years and over - Bachelor's degree,Female - AGE BY EDUCATIONAL ATTAINMENT - Population 25 years and over - Graduate or professional degree,Female - MEDIAN EARNINGS IN THE PAST 12 MONTHS (IN 2019 INFLATION-ADJUSTED DOLLARS) - Population 25 years and over with earnings - Bachelor's degree,Female - MEDIAN EARNINGS IN THE PAST 12 MONTHS (IN 2019 INFLATION-ADJUSTED DOLLARS) - Population 25 years and over with earnings - Graduate or professional degree,Total population - Sex ratio (males per 100 females),Female Ratio,Total Female Population - 25 years and over
0,Astor Pl,http://web.mta.info/nyct/service/,4-6-6 Express,1,"4 nights, 6-all times, 6 Express-weekdays AM s...",POINT (-73.99107 40.73005),145,36,061,005700,...,2356,2106,125268,503,623,102321,145240,86.2,0.537057,1265.306122
400,8th St - NYU,http://web.mta.info/nyct/service/,N-Q-R-W,401,"N-all times, Q-nights, R-all times exc nights",POINT (-73.99251 40.73046),145,36,061,005700,...,2356,2106,125268,503,623,102321,145240,86.2,0.537057,1265.306122
1,Canal St,http://web.mta.info/nyct/service/,4-6-6 Express,2,"4 nights, 6-all times, 6 Express-weekdays AM s...",POINT (-74.00019 40.71880),139,36,061,004500,...,757,452,90167,138,94,50417,101696,117.8,0.459137,347.566575
416,Canal St,http://web.mta.info/nyct/service/,R-W,417,"N-nights, R-all other times",POINT (-74.00183 40.71947),139,36,061,004500,...,757,452,90167,138,94,50417,101696,117.8,0.459137,347.566575
434,Canal St,http://web.mta.info/nyct/service/,N-Q,435,"N-all times, Q-weekdays and evenings",POINT (-74.00105 40.71881),139,36,061,004500,...,757,452,90167,138,94,50417,101696,117.8,0.459137,347.566575
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
467,125th St,http://web.mta.info/nyct/service/,4-5-6-6 Express,468,"4,6-all times, 5-all times exc nights, 6 Expre...",POINT (-73.93759 40.80414),232,36,061,019600,...,3278,544,42541,47,125,15469,75795,124.1,0.446229,1462.739848
469,34th St - Hudson Yards,http://web.mta.info/nyct/service/,7-7 Express,470,"7-all times, 7 Express-rush hours AM westbound...",POINT (-74.00220 40.75545),172,36,061,009900,...,4995,3991,113702,968,736,84800,124760,123.5,0.447427,2234.899329
470,72nd St,http://web.mta.info/nyct/service/,Q,641,Q-all times,POINT (-73.95836 40.76880),57,36,061,012600,...,10346,7228,85911,1645,2234,43722,75166,81.6,0.550661,5697.136564
471,86th St,http://web.mta.info/nyct/service/,Q,642,Q-all times,POINT (-73.95177 40.77786),197,36,061,014601,...,3734,3164,92485,741,792,84514,85992,103.2,0.492126,1837.598425


In [78]:
spatialjoin.to_pickle('Stations_W_Data.pkl')